# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [44]:
# import libraries
import pandas as pd
import numpy as np
import nltk
import re
import pickle

from sqlalchemy import create_engine

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lindstaedt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lindstaedt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lindstaedt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('Messages', engine)
X = df.message
Y = df.drop(['message', 'original', 'genre'], axis=1)

In [3]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
category_names = list(Y.columns)
print(category_names)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # build multi-language list of stop words
    stop_words = [*stopwords.words("english"),
                  *stopwords.words("french"),
                  *stopwords.words("spanish")]
    
    # lemmatize andremove stop words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word).strip() for word in tokens if word not in stop_words]
    
    return tokens

In [6]:
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [7]:
# initialize tf-idf vectorizer object
vectorizer = TfidfVectorizer(tokenizer=tokenize)

In [8]:
# compute bag of word counts and tf-idf values
XTFIDF = vectorizer.fit_transform(X.head())

In [9]:
vectorizer.vocabulary_

{'weather': 29,
 'update': 28,
 'cold': 2,
 'front': 9,
 'cuba': 6,
 'could': 3,
 'pas': 18,
 'haiti': 11,
 'hurricane': 13,
 'looking': 15,
 'someone': 23,
 'name': 16,
 'report': 19,
 'leogane': 14,
 '80': 0,
 '90': 1,
 'destroyed': 8,
 'hospital': 12,
 'st': 24,
 'croix': 5,
 'functioning': 10,
 'need': 17,
 'supply': 25,
 'desperately': 7,
 'say': 21,
 'west': 30,
 'side': 22,
 'rest': 20,
 'country': 4,
 'today': 26,
 'tonight': 27}

In [10]:
# convert sparse matrix to numpy array to view
XTFIDF.toarray()

array([[0.        , 0.        , 0.36152912, 0.36152912, 0.        ,
        0.        , 0.36152912, 0.        , 0.        , 0.36152912,
        0.        , 0.29167942, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.36152912, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.36152912, 0.36152912,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
      

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('multiOutClf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC()))),
])

In [31]:
pipeline.get_params()

{'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x0000028CC60D28C8>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('multiOutClf',
   MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
        intercept_scaling=1, loss='squared_hinge', max_iter=1000,
        multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
        verbose=0),
             n_jobs=None),
              n_jobs=None))],
 'vectorizer': CountVectorizer(analyzer='word', binar

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [13]:
y_train.related.value_counts()

1    16097
0     4874
Name: related, dtype: int64

In [14]:
pipeline.fit(X_train, y_train)

C:\Users\Lindstaedt\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
y_pred = pipeline.predict(X_test)

In [16]:
y_test.shape

(5243, 36)

In [17]:
y_pred.shape

(5243, 36)

In [28]:
for col in range(36):
    print(y_test.columns[col])
    print(classification_report(y_test.iloc[:,col], y_pred[:,col]))
    print('-----------------------------------------------------')
    

related
              precision    recall  f1-score   support

           0       0.65      0.54      0.59      1247
           1       0.86      0.91      0.89      3996

   micro avg       0.82      0.82      0.82      5243
   macro avg       0.76      0.73      0.74      5243
weighted avg       0.81      0.82      0.82      5243

-----------------------------------------------------
request
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      4384
           1       0.75      0.60      0.67       859

   micro avg       0.90      0.90      0.90      5243
   macro avg       0.84      0.78      0.81      5243
weighted avg       0.90      0.90      0.90      5243

-----------------------------------------------------
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        25

   micro avg       1.00      1.00      1.00      524

### 6. Improve your model
Use grid search to find better parameters. 

In [47]:
parameters = {
    'tfidf__use_idf':(True, False),
    'multiOutClf__estimator__estimator__class_weight': ['balanced',None],
     'multiOutClf__estimator__estimator__C':  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
     'multiOutClf__estimator__estimator__fit_intercept': [True, False],
     'multiOutClf__estimator__estimator__max_iter': [10, 100, 500],
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, scoring='accuracy', n_jobs=-1, verbose=6)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 240 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 39.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 166.1min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 385.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 701.7min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 790.0min finished
C:\Users\Lindstaedt\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'tfidf__use_idf': (True, False), 'multiOutClf__estimator__estimator__class_weight': ['balanced', None], 'multiOutClf__estimator__estimator__C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'multiOutClf__estimator__estimator__fit_intercept': [True, False], 'multiOutClf__estimator__estimator__max_iter': [10, 100, 500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=2)

In [48]:
cv.best_params_

{'multiOutClf__estimator__estimator__C': 1,
 'multiOutClf__estimator__estimator__class_weight': None,
 'multiOutClf__estimator__estimator__fit_intercept': True,
 'multiOutClf__estimator__estimator__max_iter': 100,
 'tfidf__use_idf': False}

In [49]:
cv.predict(X_test)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [60]:
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.86      0.91      0.89      3996
               request       0.75      0.60      0.67       859
                 offer       0.00      0.00      0.00        25
           aid_related       0.72      0.68      0.70      2137
          medical_help       0.55      0.27      0.36       408
      medical_products       0.65      0.30      0.41       269
     search_and_rescue       0.54      0.15      0.23       134
              security       0.14      0.01      0.02        99
              military       0.63      0.34      0.44       164
           child_alone       0.00      0.00      0.00         0
                 water       0.80      0.59      0.68       340
                  food       0.80      0.70      0.74       568
               shelter       0.75      0.56      0.64       428
              clothing       0.69      0.39      0.50        84
                 money       0.60      

In [52]:
accuracy_score(y_test, y_pred)

0.2857142857142857

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [53]:
pickle.dump(cv, open(',/model/best_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.